<a href="https://colab.research.google.com/github/manvgom/Pickioset/blob/main/SortOutput.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!rm -rf SortOutput
!mkdir SortOutput
!mkdir /content/SortOutput/dataset
!mkdir /content/SortOutput/dataset/kds_dataset
!mkdir /content/SortOutput/dataset/coco_classes
!mkdir /content/SortOutput/dataset/configuration
!cp '/content/drive/MyDrive/MTDI_TFM/Labeling.zip' '/content/SortOutput'
!unzip '/content/SortOutput/Labeling.zip' -d '/content/SortOutput'
!rm -rf /content/SortOutput/Labeling.zip /content/SortOutput/__MACOSX

import os
dirlist = os.listdir('/content/SortOutput/Labeling/true_dataset_nevera')
dirlist.remove('.DS_Store')
dirlist

for dirname in dirlist:
  %cp /content/SortOutput/Labeling/true_dataset_nevera/{dirname}/* /content/SortOutput/dataset/kds_dataset

In [92]:
import glob
with open(glob.glob('/content/SortOutput/Labeling/*.txt')[0]) as f:
    classlist = f.read().splitlines()

coco_classes = str(classlist)
coco_classes = coco_classes.replace("[", "(")
coco_classes = coco_classes.replace("]", ")")
coco_classes = 'COCO_CLASSES ='+coco_classes

with open('/content/SortOutput/dataset/coco_classes/coco_classes.py', 'w') as f:
    f.write(coco_classes)